# RMIA transformations evaluation

Put your leakpro results in the *data* folder using the following structure

data/rmia_transformations/*dataset*/*target_model*/.../result.json

Example:

data/rmia_transformations/ELD/LSTM/LiRA-num_shadow_model=64-signal_name=RescaledSMAPELoss.../result.json


In [1]:
from utils import objects_to_ablations
import glob, os, json, gc

In [2]:
data_folder = "data/rmia_transformations"
output_folder = "output/rmia_transformations"

# This is the variable that varies and we want to observe the effect of
important_name = "transformation_name"

# This is the parameters given in the structure of the data
from_path = ["dataset", "target_model"]

# The parameters given in config in result.json
from_config = ["online", "audit", "signal_name", "transformation_name"]

config = {
    "ds_indivs": {
        "EEG": 32,
        "ELD": 32,
    },
    "indiv_strategy": "indiv_mean",
}



In [3]:
objects = []

for path in glob.glob(data_folder + "/**/result.json", recursive=True):
    # Add parameters from path
    data_folder_split = os.path.join(data_folder).split(os.sep)
    path_split = path.split(os.sep)[len(data_folder_split):]
    parameters = {param: path_split[i] for i, param in enumerate(from_path)}

    # Load json file
    with open(path) as f:
        data = json.load(f)

    # Add parameters from config in result.json
    parameters |= {k: v for k,v in data["config"].items() if k in from_config}

    # Assume attack name to be part of parameters
    parameters |= {"attack": data["result_name"]}

    important_value = parameters[important_name]
    parameters = {k: v for k,v in parameters.items() if k!=important_name}

    objects.append((data, parameters, important_value))
    print((parameters, important_value))


({'dataset': 'ELD', 'target_model': 'NHiTS', 'signal_name': 'MSELoss', 'online': True, 'attack': 'RMIA'}, 'arctan')
({'dataset': 'ELD', 'target_model': 'NHiTS', 'signal_name': 'MSELoss', 'online': True, 'attack': 'RMIA'}, 'recip')
({'dataset': 'ELD', 'target_model': 'NHiTS', 'signal_name': 'MSELoss', 'online': True, 'attack': 'RMIA'}, 'reciplog')
({'dataset': 'ELD', 'target_model': 'NHiTS', 'signal_name': 'SeasonalityLoss', 'online': True, 'attack': 'RMIA'}, 'arctan')
({'dataset': 'ELD', 'target_model': 'NHiTS', 'signal_name': 'SeasonalityLoss', 'online': True, 'attack': 'RMIA'}, 'recip')
({'dataset': 'ELD', 'target_model': 'NHiTS', 'signal_name': 'SeasonalityLoss', 'online': True, 'attack': 'RMIA'}, 'reciplog')
({'dataset': 'ELD', 'target_model': 'NHiTS', 'signal_name': 'SMAPELoss', 'online': True, 'attack': 'RMIA'}, 'arctan')
({'dataset': 'ELD', 'target_model': 'NHiTS', 'signal_name': 'SMAPELoss', 'online': True, 'attack': 'RMIA'}, 'recip')
({'dataset': 'ELD', 'target_model': 'NHiTS'

In [4]:

ablations = objects_to_ablations(objects, config)

for study in ablations:
    save_dir = "-".join(f"{k}={v}" for k, v in study.parameters.items())
    save_dir = os.path.join(output_folder, save_dir)
    os.makedirs(save_dir, exist_ok=True) 
    study.make_roc_plot(save_dir)
    study.make_table(save_dir)
    gc.collect() # Garbage collect
    print(f"Saved study to {save_dir}")

Saved study to output/rmia_transformations/dataset=ELD-target_model=NHiTS-signal_name=MSELoss-online=True-attack=RMIA
Saved study to output/rmia_transformations/dataset=ELD-target_model=NHiTS-signal_name=SeasonalityLoss-online=True-attack=RMIA
Saved study to output/rmia_transformations/dataset=ELD-target_model=NHiTS-signal_name=SMAPELoss-online=True-attack=RMIA
Saved study to output/rmia_transformations/dataset=ELD-target_model=NHiTS-signal_name=TrendLoss-online=True-attack=RMIA


<Figure size 640x480 with 0 Axes>